In [1]:
import json
from datasets import Dataset, concatenate_datasets
import torch
import numpy as np
from nltk.tokenize import sent_tokenize
import nltk
# nltk.download('punkt')
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

/home/ppibot/anaconda3/envs/myenv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

In [3]:
with open('../input/parser_training/parser_training_data.json', 'r') as f:
    training_data_a = json.load(f)

In [4]:
useful_tags = ['TSE', 'TEE', 'TBE', 'CE','AttributeName', 'AttributeValue', 'AGR', 'GBC', 'FDE']

cleaned_training_data = []
for phrase in training_data_a['data']:
    useful_slots = []
    for slots in phrase['annotation']:
        if slots['tag'] in useful_tags:
            useful_slots.append(slots)
    if len(useful_slots) > 0:
        cleaned_training_data.append({'text': phrase['description'], 'slots': useful_slots})

In [5]:
with open('../input/metrics_dataset.json', 'r') as f:
    training_data_b = json.load(f)

In [6]:
with open('../input/paraphrase_data.json', 'r') as f:
    paraphrase_data = json.load(f)

In [7]:
cleaned_training_data_aux = []
for phrase in training_data_b["metrics"]:
    useful_slots = []
    local_slots = []
    for slots in phrase['slots']:
        if slots in useful_tags:
            local_dict = {}
            local_dict['tag'] = slots
            local_dict['text'] = phrase['slots'][slots]
            local_slots.append(local_dict)
    if len(local_slots) > 0:
        cleaned_training_data_aux.append({'text': phrase['description'], 'slots': local_slots})

print(cleaned_training_data_aux)

[{'text': 'Number of incidents that are not resolved', 'slots': [{'tag': 'CE', 'text': 'incidents that are not resolved'}]}, {'text': 'Average time from opened to closed', 'slots': [{'tag': 'AGR', 'text': 'Average'}, {'tag': 'TSE', 'text': 'opened'}, {'tag': 'TEE', 'text': 'closed'}]}, {'text': 'Average real time from opening to resolution of the incident', 'slots': [{'tag': 'AGR', 'text': 'Average'}, {'tag': 'TSE', 'text': 'opening'}, {'tag': 'TEE', 'text': 'resolution of the incident'}]}, {'text': 'Average number of priority changes', 'slots': [{'tag': 'AGR', 'text': 'Average'}, {'tag': 'CE', 'text': 'priority changes'}]}, {'text': 'Average delays caused by appealing to prefacture', 'slots': [{'tag': 'AGR', 'text': 'Average'}, {'tag': 'TBE', 'text': 'appealing to prefacture'}]}, {'text': 'Average number of fines that finish without payment', 'slots': [{'tag': 'AGR', 'text': 'Average'}, {'tag': 'CE', 'text': 'fines that finish without payment'}]}, {'text': 'Average number of payments 

In [8]:
cleaned_training_data_aux_2 = []
for phrase in paraphrase_data["metrics"]:
    useful_slots = []
    local_slots = []
    for slots in phrase['slots']:
        if slots in useful_tags:
            local_dict = {}
            local_dict['tag'] = slots
            local_dict['text'] = phrase['slots'][slots]
            local_slots.append(local_dict)
    if len(local_slots) > 0:
        cleaned_training_data_aux_2.append({'text': phrase['description'], 'slots': local_slots})

print(cleaned_training_data_aux_2)

[{'text': 'The average time from the opening to the resolution of the incident', 'slots': [{'tag': 'AGR', 'text': 'average'}, {'tag': 'TSE', 'text': 'the opening'}, {'tag': 'TEE', 'text': 'the resolution of the incident'}]}, {'text': 'The mean duration from the commencement to the conclusion of the event', 'slots': [{'tag': 'AGR', 'text': 'mean'}, {'tag': 'TSE', 'text': 'the commencement'}, {'tag': 'TEE', 'text': 'the conclusion of the event'}]}, {'text': 'The average time from customer inquiry to response', 'slots': [{'tag': 'AGR', 'text': 'average'}, {'tag': 'TSE', 'text': 'customer inquiry'}, {'tag': 'TEE', 'text': 'response'}]}, {'text': 'The average time from purchase confirmation to delivery', 'slots': [{'tag': 'AGR', 'text': 'average'}, {'tag': 'TSE', 'text': 'purchase confirmation'}, {'tag': 'TEE', 'text': 'delivery'}]}, {'text': 'The average time from task assignment to completion', 'slots': [{'tag': 'AGR', 'text': 'average'}, {'tag': 'TSE', 'text': 'task assignment'}, {'tag':

In [9]:
cleaned_training_data.extend(cleaned_training_data_aux)

In [10]:
cleaned_training_data.extend(cleaned_training_data_aux_2)

In [11]:
inputs = []
targets = []

for phrase in cleaned_training_data:
    prompt = f"Sentence: {phrase['text']}\nAvailable Tags: {', '.join(useful_tags)}"
    target = '; '.join([f"{slot['text']}: {slot['tag']}" for slot in phrase['slots']])
    inputs.append(prompt)
    targets.append(target)

print(inputs)
print(targets)

['Sentence: Elapsed time between an action requirement is notified until the action is performed.\nAvailable Tags: TSE, TEE, TBE, CE, AttributeName, AttributeValue, AGR, GBC, FDE', 'Sentence: Elapsed time between action is performed until the corresponding documentation is delivered.\nAvailable Tags: TSE, TEE, TBE, CE, AttributeName, AttributeValue, AGR, GBC, FDE', 'Sentence: the elapsed time between the notification to contractor of the installation requirement and its planning.\nAvailable Tags: TSE, TEE, TBE, CE, AttributeName, AttributeValue, AGR, GBC, FDE', 'Sentence: the elapsed time between APC contracts the service and contractor is able to start it.\nAvailable Tags: TSE, TEE, TBE, CE, AttributeName, AttributeValue, AGR, GBC, FDE', 'Sentence: the elapsed time between the end and closure of the installation and the documentation submission\nAvailable Tags: TSE, TEE, TBE, CE, AttributeName, AttributeValue, AGR, GBC, FDE', 'Sentence: the elapsed time between repair notification and

In [12]:
examples = {
    "sentence": inputs,
    "label": targets
}

In [13]:
dataset = Dataset.from_dict(examples).train_test_split(test_size=0.2)

In [14]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = sample["sentence"]

    # tokenize inputs
    model_inputs = tokenizer(inputs, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["label"], padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [15]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

In [16]:
from hermetrics.levenshtein import Levenshtein

def compute_slot_metric(preds, real):
    phrases_metrics = []
    for real_phrase, pred_phrase in zip(real,preds):
        real_slots = real_phrase.split("; ") 
        pred_slots = pred_phrase.split("; ")
        slots_metrics = []
        for real_slot in real_slots:
            real_tag = real_slot.split(": ")[1]
            slot_metric = 0
            for pred_slot in pred_slots:
                try:
                    pred_tag = pred_slot.split(": ")[1]
                    if real_tag == pred_tag:
                        real_text = real_slot.split(": ")[0]
                        pred_text = pred_slot.split(": ")[0]
                        slot_metric = (1 - Levenshtein().normalized_distance(real_text, pred_text))
                except Exception as e:
                    pass
            slots_metrics.append(slot_metric)
        phrases_metrics.append(np.mean(slots_metrics))
    return np.mean(phrases_metrics)
    

In [17]:
def compute_slot_accuracy(preds, real):
    phrases_metrics = []
    for real_phrase, pred_phrase in zip(real,preds):
        real_slots = real_phrase.split("; ") 
        pred_slots = pred_phrase.split("; ")
        slots_metrics = []
        for real_slot in real_slots:
            real_tag = real_slot.split(": ")[1]
            slot_metric = 0
            for pred_slot in pred_slots:
                try:
                    pred_tag = pred_slot.split(": ")[1]
                    if real_tag == pred_tag:
                        real_text = real_slot.split(": ")[0]
                        pred_text = pred_slot.split(": ")[0]
                        if real_text == pred_text:
                            slot_metric = 1
                except Exception as e:
                    pass
            slots_metrics.append(slot_metric)
        phrases_metrics.append(np.mean(slots_metrics))
    return np.mean(phrases_metrics)

In [18]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # compute slot metric
    slot_metric = compute_slot_metric(decoded_preds, decoded_labels)
    slot_accuracy = compute_slot_accuracy(decoded_preds, decoded_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(decoded_labels, decoded_preds, average='macro')
    acc = accuracy_score(decoded_labels, decoded_preds)
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall,
        "slot": slot_metric,
        "slot_accuracy": slot_accuracy
    }

In [19]:
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["sentence"], truncation=True), batched=True, remove_columns=["sentence", "label"])
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["label"], truncation=True), batched=True, remove_columns=["sentence", "label"])
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["label", "sentence"])

# device = torch.device("cuda")
# model = model.to(device)

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)
'''# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir= f"./model",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=10,
    # logging & evaluation strategies
    evaluation_strategy="epoch",
    save_strategy="no",
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    push_to_hub=False
)
# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)
# Train model
trainer.train()
'''

100%|██████████| 1/1 [00:00<00:00, 85.09ba/s]


'# Define training args\ntraining_args = Seq2SeqTrainingArguments(\n    output_dir= f"./model",\n    per_device_train_batch_size=2,\n    per_device_eval_batch_size=2,\n    predict_with_generate=True,\n    fp16=False, # Overflows with fp16\n    learning_rate=5e-5,\n    num_train_epochs=10,\n    # logging & evaluation strategies\n    evaluation_strategy="epoch",\n    save_strategy="no",\n    # metric_for_best_model="overall_f1",\n    # push to hub parameters\n    push_to_hub=False\n)\n# Create Trainer instance\ntrainer = Seq2SeqTrainer(\n    model=model,\n    args=training_args,\n    train_dataset=tokenized_dataset["train"],\n    eval_dataset=tokenized_dataset["test"],\n    compute_metrics=compute_metrics,\n    data_collator=data_collator,\n)\n# Train model\ntrainer.train()\n'

In [20]:
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

In [21]:
training_args = Seq2SeqTrainingArguments(
    output_dir= f"./model",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=10,
    # logging & evaluation strategies
    evaluation_strategy="epoch",
    save_strategy="epoch",
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="slot_accuracy",
    push_to_hub=False
)

In [22]:
trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

loading configuration file config.json from cache at /home/ppibot/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/c782cba52f8ea6a704240578055cf1c3fc2f2ca9/config.json
Model config T5Config {
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length"

In [23]:
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-4, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 20),
    }

In [24]:
def compute_objective(metrics):
    return metrics["eval_slot_accuracy"]

In [25]:
best_run = trainer.hyperparameter_search(
    n_trials=50,
    direction="maximize",
    hp_space=optuna_hp_space,
    compute_objective=compute_objective,
)

[I 2023-06-11 18:31:58,117] A new study created in memory with name: no-name-8140c2c7-1e5b-4faa-afda-c40a438fd6e4
Trial: {'learning_rate': 3.075754023268171e-05, 'num_train_epochs': 14}
loading configuration file config.json from cache at /home/ppibot/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/c782cba52f8ea6a704240578055cf1c3fc2f2ca9/config.json
Model config T5Config {
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "rela

{'eval_loss': 0.6226199269294739, 'eval_accuracy': 0.05405405405405406, 'eval_f1': 0.02857142857142857, 'eval_precision': 0.02857142857142857, 'eval_recall': 0.02857142857142857, 'eval_slot': 0.30389227579097716, 'eval_slot_accuracy': 0.1554054054054054, 'eval_runtime': 2.6084, 'eval_samples_per_second': 14.185, 'eval_steps_per_second': 7.284, 'epoch': 1.0}


Model weights saved in ./model/run-0/checkpoint-73/pytorch_model.bin
 14%|█▍        | 146/1022 [00:33<02:49,  5.17it/s]***** Running Evaluation *****
  Num examples = 37
  Batch size = 2
/home/ppibot/anaconda3/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ppibot/anaconda3/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
                                                  
 14%|█▍        | 146/1022 [00:36<02:49,  5.17it/s]Saving model checkpoint to ./model/run-0/checkpoint-

{'eval_loss': 0.4684683680534363, 'eval_accuracy': 0.2702702702702703, 'eval_f1': 0.15873015873015872, 'eval_precision': 0.15873015873015872, 'eval_recall': 0.15873015873015872, 'eval_slot': 0.6071956660128858, 'eval_slot_accuracy': 0.554054054054054, 'eval_runtime': 2.7213, 'eval_samples_per_second': 13.597, 'eval_steps_per_second': 6.982, 'epoch': 2.0}


Model weights saved in ./model/run-0/checkpoint-146/pytorch_model.bin
 21%|██▏       | 219/1022 [00:52<02:35,  5.17it/s]***** Running Evaluation *****
  Num examples = 37
  Batch size = 2
/home/ppibot/anaconda3/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ppibot/anaconda3/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
                                                  
 21%|██▏       | 219/1022 [00:55<02:35,  5.17it/s]Saving model checkpoint to ./model/run-0/checkpoint

...

{'eval_loss': 0.32394325733184814, 'eval_accuracy': 0.6216216216216216, 'eval_f1': 0.45098039215686275, 'eval_precision': 0.45098039215686275, 'eval_recall': 0.45098039215686275, 'eval_slot': 0.8823244370374589, 'eval_slot_accuracy': 0.7972972972972973, 'eval_runtime': 2.8584, 'eval_samples_per_second': 12.945, 'eval_steps_per_second': 6.647, 'epoch': 3.0}


Model weights saved in ./model/run-48/checkpoint-219/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./model/run-48/checkpoint-219 (score: 0.7972972972972973).
100%|██████████| 219/219 [01:02<00:00,  3.53it/s]
[I 2023-06-11 20:30:20,068] Trial 48 finished with value: 0.7972972972972973 and parameters: {'learning_rate': 9.060355355910528e-05, 'num_train_epochs': 3}. Best is trial 11 with value: 0.8558558558558559.
Trial: {'learning_rate': 6.944402807070206e-05, 'num_train_epochs': 19}


{'train_runtime': 62.0539, 'train_samples_per_second': 7.058, 'train_steps_per_second': 3.529, 'train_loss': 0.5389401161507384, 'epoch': 3.0}


loading configuration file config.json from cache at /home/ppibot/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/c782cba52f8ea6a704240578055cf1c3fc2f2ca9/config.json
Model config T5Config {
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length"

{'eval_loss': 0.4376596510410309, 'eval_accuracy': 0.24324324324324326, 'eval_f1': 0.13846153846153847, 'eval_precision': 0.13846153846153847, 'eval_recall': 0.13846153846153847, 'eval_slot': 0.6567216584601709, 'eval_slot_accuracy': 0.47297297297297297, 'eval_runtime': 3.0407, 'eval_samples_per_second': 12.168, 'eval_steps_per_second': 6.249, 'epoch': 1.0}


In [26]:
best_run

BestRun(run_id='11', objective=0.8558558558558559, hyperparameters={'learning_rate': 9.450559422241211e-05, 'num_train_epochs': 15})

In [27]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()
































loading configuration file config.json from cache at /home/ppibot/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/c782cba52f8ea6a704240578055cf1c3fc2f2ca9/config.json
Model config T5Config {
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length"

{'eval_loss': 0.36733299493789673, 'eval_accuracy': 0.3783783783783784, 'eval_f1': 0.23333333333333334, 'eval_precision': 0.23333333333333334, 'eval_recall': 0.23333333333333334, 'eval_slot': 0.7315882489581004, 'eval_slot_accuracy': 0.6531531531531531, 'eval_runtime': 2.9604, 'eval_samples_per_second': 12.498, 'eval_steps_per_second': 6.418, 'epoch': 1.0}


Model weights saved in ./model/checkpoint-73/pytorch_model.bin
 13%|█▎        | 146/1095 [00:34<03:09,  5.01it/s]***** Running Evaluation *****
  Num examples = 37
  Batch size = 2
/home/ppibot/anaconda3/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ppibot/anaconda3/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
                                                  
 13%|█▎        | 146/1095 [00:37<03:09,  5.01it/s]Saving model checkpoint to ./model/checkpoint-146
Configur

{'eval_loss': 0.35149717330932617, 'eval_accuracy': 0.4864864864864865, 'eval_f1': 0.32142857142857145, 'eval_precision': 0.32142857142857145, 'eval_recall': 0.32142857142857145, 'eval_slot': 0.7397442373418539, 'eval_slot_accuracy': 0.6846846846846847, 'eval_runtime': 2.9525, 'eval_samples_per_second': 12.532, 'eval_steps_per_second': 6.435, 'epoch': 2.0}


Model weights saved in ./model/checkpoint-146/pytorch_model.bin
 20%|██        | 219/1095 [00:55<02:54,  5.01it/s]***** Running Evaluation *****
  Num examples = 37
  Batch size = 2
/home/ppibot/anaconda3/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ppibot/anaconda3/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
                                                  
 20%|██        | 219/1095 [00:57<02:54,  5.01it/s]Saving model checkpoint to ./model/checkpoint-219
Configu

...

Model weights saved in ./model/checkpoint-1022/pytorch_model.bin
100%|██████████| 1095/1095 [05:08<00:00,  5.03it/s]***** Running Evaluation *****
  Num examples = 37
  Batch size = 2
/home/ppibot/anaconda3/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ppibot/anaconda3/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
                                                   
100%|██████████| 1095/1095 [05:11<00:00,  5.03it/s]Saving model checkpoint to ./model/checkpoint-1095
Co

{'eval_loss': 0.41605108976364136, 'eval_accuracy': 0.7027027027027027, 'eval_f1': 0.5416666666666666, 'eval_precision': 0.5416666666666666, 'eval_recall': 0.5416666666666666, 'eval_slot': 0.9227994430297062, 'eval_slot_accuracy': 0.8558558558558559, 'eval_runtime': 3.0394, 'eval_samples_per_second': 12.173, 'eval_steps_per_second': 6.251, 'epoch': 15.0}


Model weights saved in ./model/checkpoint-1095/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./model/checkpoint-949 (score: 0.8558558558558559).
100%|██████████| 1095/1095 [05:15<00:00,  3.47it/s]

{'train_runtime': 315.4531, 'train_samples_per_second': 6.942, 'train_steps_per_second': 3.471, 'train_loss': 0.1310899735041405, 'epoch': 15.0}


TrainOutput(global_step=1095, training_loss=0.1310899735041405, metrics={'train_runtime': 315.4531, 'train_samples_per_second': 6.942, 'train_steps_per_second': 3.471, 'train_loss': 0.1310899735041405, 'epoch': 15.0})

In [28]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 37
  Batch size = 2
100%|██████████| 19/19 [00:02<00:00,  6.71it/s]/home/ppibot/anaconda3/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ppibot/anaconda3/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
100%|██████████| 19/19 [00:02<00:00,  6.57it/s]


{'eval_loss': 0.42054837942123413,
 'eval_accuracy': 0.7027027027027027,
 'eval_f1': 0.5416666666666666,
 'eval_precision': 0.5416666666666666,
 'eval_recall': 0.5416666666666666,
 'eval_slot': 0.9227994430297062,
 'eval_slot_accuracy': 0.8558558558558559,
 'eval_runtime': 3.0483,
 'eval_samples_per_second': 12.138,
 'eval_steps_per_second': 6.233,
 'epoch': 15.0}

In [29]:
import time 
now = time.strftime("%Y-%m-%d %H:%M", time.localtime())
trainer.save_model(f"./parsing-model-{now}")

Saving model checkpoint to ./parsing-model-2023-06-11 20:35
Configuration saved in ./parsing-model-2023-06-11 20:35/config.json
Model weights saved in ./parsing-model-2023-06-11 20:35/pytorch_model.bin
